In [ ]:
# Brief: import common libraries and frameworks
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from utilities3 import LpLoss
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import math
from sklearn.model_selection import train_test_split
from functools import reduce
from functools import partial
import operator
from timeit import default_timer
from matplotlib.ticker import FormatStrFormatter
import deepxde as dde
# Import kernel solver functions from external module
from kernel_solvers import solver_2x2, K_solver_2x2

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
import numpy as np
from scipy.integrate import odeint
from scipy.interpolate import pchip_interpolate
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.integrate import trapz

# from scipy import sparse
from math import atan2, pi, sin, cos, sqrt

## cal system parameters and funcions

In [5]:
def cal_syspara(rs):
    class Struct:
        pass

# System configuration
    sys = Struct()
    sys.ctrl_on = 1
    sys.RGB_color1 = [1, 0, 0]
    sys.RGB_color2 = [0, 0, 1]
    sys.RGB_color3 = [0, 1, 0]
    sys.style1 = '--'
    sys.style2 = '-.'
    sys.style3 = '-'

    # Simulation parameters
    # Parameter values
    vm = 40  # m/s 100 miles per hour
    rm = 0.16  # 240 veh/mile /m
    tau = 60  # s
    L = 500  # m 1km
    T = 240  # second /hour
    # rs = 0.12 # veh/mile
    Veq = lambda rho: vm * (1 - rho/rm)
    vs = Veq(rs)
    gamma = 1
    qs = rs * vs # per second
    ps = vm/rm * qs/vs

    # Adding to sys
    sys.vs = vs
    sys.rs = rs
    sys.ps = ps
    sys.gamma = gamma
    sys.L = L
    sys.tau = tau
    sys.qs = qs

    # System parameters
    fun = Struct()
    fun.lam = lambda x: 1*x + vs - x
    fun.mu = lambda x: 1*x + gamma*ps - vs - x
    fun.lam_d = lambda x: 1*x - x
    fun.mu_d = lambda x: 1*x - x
    fun.c_1 = lambda x: 1*x - x
    fun.c_2 = lambda x: -1/tau * np.exp(-x/(tau*vs))
    fun.q = (vs - gamma*ps)/vs
    sys.fun = fun
    sys.N = 100
    sys.N_grid = sys.N + 2
    sys.N_g = 100
    sys.simH = 300
    sys.h = 0.1
    sys.Tspan = np.arange(0, sys.simH+sys.h, sys.h)
    sys.Delta = L / (sys.N + 1)
    sys.xspan = np.linspace(0, L, sys.N_grid)
    sys.xspanT = np.arange(sys.Delta, L - sys.Delta + sys.h, sys.Delta)
    sys.intArr = sys.Delta * np.hstack((0.5, np.ones(sys.N), 0.5))
    sys.lam = fun.lam(sys.xspan)
    sys.mu = fun.mu(sys.xspan)
    sys.c_1 = fun.c_1(sys.xspan)
    sys.c_2 = fun.c_2(sys.xspan)
    sys.q = fun.q
    sys.t_1 = np.dot(sys.intArr, 1/sys.lam)
    sys.t_2 = np.dot(sys.intArr, 1/sys.mu)
    sys.t_F = sys.t_1 + sys.t_2
    sys.gamma_U = 1
    return sys, fun

In [31]:
sys, fun = cal_syspara(0.13)
print(sys.vs)
print(sys.rs)
print(sys.mu[0])

7.5
0.13
25.000000000000007


In [7]:
mu = []
Kernel1 = []
Kernel2 = []
for i in range(1):
    rs = np.random.uniform(0.09,0.13)
    sys,fun = cal_syspara(rs=rs)
    Kvu, Kvv = K_solver_2x2(fun, sys.N_g)
    Kernel1.append(Kvu)
    Kernel2.append(Kvv)
    mu.append(sys.mu)

Kernel1 = np.array(Kernel1)
Kernel2 = np.array(Kernel1)
Kernel1 = Kernel1.reshape(Kernel1.shape[0], -1)
Kernel2 = Kernel2.reshape(Kernel2.shape[0], -1)

mu = np.array(mu)
Kernel_all = np.hstack((Kernel1,Kernel2))
x= np.array(mu, dtype=np.float32)
y= np.array(Kernel_all, dtype=np.float32)

np.savetxt("x.dat", x)
np.savetxt("y.dat", y)
